# 💰 Retail Financial Analysis Dashboard
## Profitability, Efficiency, and Liquidity Analysis

**Project Context**: This dashboard translates row-level sales and inventory data into executive financial insights. We analyze the 'Health' of the business using professional auditing and accounting metrics.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Set plotting theme
import plotly.io as pio
pio.templates.default = "plotly_white"
pio.renderers.default = "notebook_connected"

print("✅ Financial visualization engine ready.")

✅ Financial visualization engine ready.


## 1. The Bottom Line: Revenue vs. Net Income
Tracking how much of our top-line 'Revenue' actually makes it to the 'Net Income' after all expenses and COGS.

In [2]:
# Load Income Statement data (it was saved with month as index)
is_path = '../outputs/retail_financial_report.xlsx'
# We'll read from the excel file we just generated or re-run the logic? 
# Better to read the excel to show it works.
is_df = pd.read_excel(is_path, sheet_name='Income Statement', index_col=0)
is_df.index = pd.to_datetime(is_df.index.astype(str))

fig = go.Figure()
fig.add_trace(go.Bar(x=is_df.index.strftime('%b %Y'), y=is_df['revenue'], name='Revenue', marker_color='#2E86C1'))
fig.add_trace(go.Bar(x=is_df.index.strftime('%b %Y'), y=is_df['Net Income'], name='Net Income', marker_color='#27AE60'))

fig.update_layout(barmode='group', title='Monthly Revenue vs. Net Income',
                  yaxis_title='Amount ($)', template='plotly_white')
fig.show()

## 2. Retail Performance Ratios (Benchmarks)
How do we compare against industry standards for retail?

In [3]:
# Load Ratios
ratios_df = pd.read_excel(is_path, sheet_name='Financial Ratios')

fig = go.Figure()

for i, row in ratios_df.iterrows():
    if row['Ratio'] in ['GMROI', 'Inventory Turnover', 'Current Ratio']:
        fig.add_trace(go.Indicator(
            mode = "number+gauge+delta",
            value = row['Value'],
            delta = {'reference': row['Benchmark']},
            domain = {'row': 0, 'column': i},
            title = {'text': row['Ratio']},
            gauge = {
                'axis': {'range': [None, row['Benchmark']*2]},
                'bar': {'color': "#2E86C1"},
                'steps': [
                    {'range': [0, row['Benchmark']], 'color': "#FADBD8"},
                    {'range': [row['Benchmark'], row['Benchmark']*2], 'color': "#D5F5E3"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': row['Benchmark']
                }
            }
        ))

fig.update_layout(grid = {'rows': 1, 'columns': 3, 'pattern': "independent"},
                  height=400, title_text="Financial Health Indicators vs. Benchmarks")
fig.show()

## 3. Margin Analysis
A sustainable business needs stable margins. Variations in Net Margin usually signal rising operating costs.

In [4]:
fig = px.line(is_df, y=['Gross Margin %', 'Net Margin %'], 
              title='Profitability Margins Trend',
              labels={'value': 'Percentage (%)', 'index': 'Month'},
              markers=True)
fig.update_layout(yaxis_range=[0, 100])
fig.show()

## 4. Executive Summary

1. **Profitability**: Our business maintains a healthy **Gross Margin (~45%)**, but Net Margin is sensitive to high **OpEx** (~19% of revenue).
2. **Inventory Efficiency**: The **GMROI** is extremely high, suggesting we are getting significant profit for every dollar of stock. However, a high turnover might also mean we are running too lean and risking stockouts.
3. **Liquidity Warning**: The **Current Ratio** is below the benchmark of 2.0. We should focus on improving our cash position to ensure short-term stability.